<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_27%ED%8C%8C%EC%9D%B4%EC%8D%AC(%EB%AA%A8%ED%98%95%EC%9D%98_%EC%84%A0%ED%98%95%EC%84%B1%2C_%EC%9E%94%EC%B0%A8%EC%9D%98_%EC%A0%95%EA%B7%9C%EC%84%B1%2C_%EC%9E%94%EC%B0%A8%EC%9D%98_%EB%93%B1%EB%B6%84%EC%82%B0%EC%84%B1%2C_%EC%9E%94%EC%B0%A8%EC%9D%98_%EB%8F%85%EB%A6%BD%EC%84%B1%2C_%EA%B7%B9%EB%8B%A8%EA%B0%92%2C_%EB%8B%A4%EC%A4%91%EA%B3%B5%EC%84%A0%EC%84%B1%2C_VIF%2C_LogisticRegression%2C_%EC%8B%9C%EA%B7%B8%EB%AA%A8%EC%9D%B4%EB%93%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols


boston = pd.read_csv('c:/data/boston.csv')
boston.head()
boston.info()

col = boston[boston.columns.difference(['MEDV'])].columns
ols_lr = ols('MEDV ~ '+' + '.join(col),data=boston).fit()
ols_lr.summary()


★ 모형의 선형성
- 예측값(predicted)과 잔차(residual) 비교  # 잔차 : 예측값과 실제값의 차이
- 모든 예측값에서 잔차가 비슷하게 있어야한다. 
- 빨간 실선은 잔차의 추세를 나타내는 선
- 빨간 실선이 점선에서 크게 벗어나면 예측값에 따라 잔차가 크게 달라진다는 것을 의미

x = boston[boston.columns.difference(['MEDV'])]
y = boston['MEDV']

predicted = ols_lr.predict(x) # 예측값
residual = y - predicted # 잔차
sns.regplot(predicted,residual, line_kws={'color':'red'})
plt.plot([predicted.min(),predicted.max()],[0,0],'--',color='grey')

★ 잔차의 정규성
- 잔차가 정규분포를 따른다는 가정
- Q-Q plot으로 확인할 수 있다. (Quantile-Quantile Plot)
- 잔차가 정규분포를 띄면 Q-Q plot에서 점들이 점선을 따라 배치되어 있어야한다.

import scipy.stats as ss
z = ss.zscore(residual)
ss.probplot(z,plot=plt) # Q-Q plot 빨간선에 가까울수록 정규성을 따른다.

sns.distplot(residual)
sns.histplot(residual)

★ 잔차의 등분산성
- 회귀모형을 통해 예측된 값이 크던, 작던 모든 값들에 대하여 잔차의 분산이 동일하는 것으로 가정
- 빨간색 실선이 수평선을 그리는 것이 가장 이상적이다.

sns.regplot(predicted,np.sqrt(np.abs(z)), line_kws={'color':'red'})

★ 잔차의 독립성
- 자료 수집 과정에서 무작위 표본을 이용해서 모델을 학습했다면 잔차의 독립성은 만족하는 것으로 본다.
- Durbin-Watson 검정을 이용해서 확인
- 보통 1.5 ~ 2.5 사이이면 독립으로 판단하고 회귀모형이 적합하다고 판단하면 된다.
- 0(양의자기상관) 또는 4(음의자기상관)에 가깝다는 것은 잔차들이 자기 상관을 가지고 있다는 의미이고 
  이는 t값,F값, R²값이 실제보다 증가시켜 유의미하지 않는 결과를 유의미한 결과로 왜곡할 수 있다.

★ 극단값
- cook`s distance는 극단값을 나타내는 지표
- 데이터가 특히 예측에서 많이 벗어남을 알 수 있다.

from statsmodels.stats.outliers_influence import OLSInfluence
cd, _ = OLSInfluence(ols_lr).cooks_distance # 여러개 값중 하나만 받겠다 : 변수명, _
cd.sort_values(ascending=False).head()

ols_lr.predict(boston.iloc[368,])
# 예측값368    23.800729
# MEDV      50.000000
# 50.00000 - 23.800729 = 잔차

ols_lr.predict(boston.iloc[200,])
# 예측200    30.643939
# 실제값MEDV   32.90000

R²(R-squared)
-종속변수의 분산을 독립변수의 분산으로 설명하는 지표
R² = 0 : 모델의 설명력이 0
R² = 1 : 모델의 설명령이 100%

R² (R-squared)
- R² 분산기반으로 예측성능을 평가한다. 1에 가까울수록 예측정확도가 높다
r2_score(y_test,y_pred)
1- np.sum(np.power(y_test - y_pred,2)) / np.sum(np.power(y_test - np.mean(y_test),2))


            SSR
R² = 1 - --------
            SST

★ 다중공선성(Multicollinearity)
- 독립변수들 끼리 상관관계를 가지고 있다.

예)
혈압(종속변수), 키, 몸무게, BMI,...
...
...

★ VIF를 통한 다중공선성 확인
- VIF는 Variance Inflation Factor 지표를 이용
- 분산팽창요인
- VIF가 10보다 크면 다중공선성이 있다고 판단한다.
- ONE HOT ENCODING의 변수는 VIF가 3 이상이면 다중공선성이 있다고 판단 # 체크해봐야하는 기준이되는 것이고 무조건 버리는 것은 아니다.

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(boston.values,i) for i in range(boston.shape[1])]
vif['features'] = boston.columns
vif

data = boston[boston.columns.difference(['NOX','PTRATIO','TAX'])]
data_vif = pd.DataFrame()
data_vif['VIF Factor'] = [variance_inflation_factor(data.values,i) for i in range(data.shape[1])]
data_vif['features'] = data.columns
data_vif

# 프로젝트할 때 막연하게 그냥 하는게 아니라 도메인 지식에 대해서 공부가 필요
    
# 상관계수로 확인(변수간의 상관계수 확인하고 불필요한 부분은 제외하기)
corr_matrix = boston.corr().round(2)
sns.heatmap(data=corr_matrix,annot=True)
# 잔차, 다중공선성, P값, R² 를 체크해서 회귀분석 프로젝트 발표할 때 정확히 이야기할 수 있어야함
# 모든피쳐를 다 넣는다고 좋은 것이 아니고 P값이 0.05보다 작은 값이 기울기가 의미가 있는 값

★ Logistic Regression(로지스틱회귀분석)
- 선형회귀방식을 분류에 적용한 알고리즘
- 분류를 하는데 있어서 가장 흔한 경우는 이분법을 기준으로 분류하는 경우, 이진분류(0,1)
- 기업부도예측, 주가, 환율, 금리등의 up/down 예측

오즈비(odds ratio)
- 오즈비는 확률과 관련된 의미로 P가 주어졌을 때 사건이 발생할 확률(성공)이 발생하지 않을 확률(실패)에 비해
  몇배 더 높은 가의 의미
- 예)
    종속변수의 범주가 1(성공), 0(실패)인 이분형을 가정할 때 P(확률)가 0.8이라면
    오즈비는 (0.8/(1-0.8)) = 4가 되고 이값은 성공이 될 확률이 실패가 될 확률보다 4배 높다는 의미
              P      성공
    odds = ------ = ------
             1-P     실패

    logit = log(odds)

# log를 사용해서 값이 이분이 안되고 무한대로 가니까 최종 분류에서는 0과 1로 분류해야하기에 시그모이드 함수를 사용한다

★ sigmoid(시그모이드)
- 0 ~ 1 사이의 실수값으로 리턴하는 함수
- e 자연상수 : 2.7182

np.exp(-10)
np.exp(10)
                   1
sigmoid(x) = --------------
             1 + np.exp(-1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

sigmoid(1000)
sigmoid(-1000)
sigmoid(100)
sigmoid(12)

x  = np.arange(-5,5,0.1) #-5 ~ 5사이 0.1씩 단위 값
y = sigmoid(x) 
plt.plot(x,y)

sigmoid(y(예측)) = ax + b
0 ~ 1 값만 출력할 수 있도록 시그모이드 함수를 이용한다.

sigmoid(y(예측)) >= 0.5 : 1
sigmoid(y(예측)) <  0.5 : 0
# 신경망에서는 기본값을 지정할 수 있지만 여기 sklearn에서 제공하는 함수에서는 0.5가 기본값이 됨

# 독립변수, 종속변수 둘 다 수치형으로 되야하는데 문자형도 자동으로 변환해주는 함수가 있다
from sklearn.linear_model import LogisticRegression
iris = pd.read_csv('c:/data/iris.csv')
x = iris.iloc[:,:-1]
y = iris.Name

log_lr = LogisticRegression()
log_lr.fit(x,y)

log_lr.classes_  #분류되는 종속변수값

log_lr.predict([[5.1,3.5,1.4,0.2]])[0] # 2차원으로 넣어야함
log_lr.predict([[6.9,3.2,5.4,2.2]])[0]

import seaborn as sns
sns.stripplot(x='PetalLength',y='Name',data=iris)
sns.stripplot(x='PetalWidth',y='Name',data=iris)
sns.stripplot(x='SepalLength',y='Name',data=iris)
sns.stripplot(x='SepalWidth',y='Name',data=iris)
sns.stripplot(y='PetalLength',x='Name',data=iris) # x, y 거꾸로

sns.swarmplot(x='PetalLength',y='Name',data=iris) # 점들이 중복(겹처서)되서 나오지 않음

# 프로젝트에서 나이브베이즈! 로지스틱회귀! 둘다 해보고 정확도를 비교해보자


# naivebayes프로젝트----------------------------------------------------------------------------------------------------------------------
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report
import pandas as pd
from pandas import DataFrame,Series
from collections import Counter
from konlpy.tag import Okt
okt = Okt()

# 데이터 읽어오기
book = pd.read_csv('c:/data/book_field1.csv',encoding='utf-8')
book

# 학습, 테스트 데이터, 학습라벨, 테스트라벨 만들기
x_train,x_test,y_train,y_test = train_test_split(book['text'],book['field'],test_size=0.2) # test_size=0.2는 전체의 20%를 test셋으로 만들겠다는 의미, shuffle : default=True split이전에 섞을건지, stratify : target으로 지정하면 각각의 class비율을 train /validation에 유지(한쪽에 쏠려서 분배되는 것을 방지함(default=None ))
Counter(y_train)
Counter(y_test)

# 전처리, 형태소 분석 함수
def okt_pos(arg):
    token = []
    for j in okt.pos(arg):
        if j[1] in ['Noun','Adjective','Alpha']:  # 명사, 형용사, 영어
            token.append(j[0]) 
    token = [i for i in token if len(i)>=2]
    return token

# 불용어
stopwords = ['지난','이후','독자','작가','있다','같은','이로','인해','있게','누구','있으며','있는','통해','있도록','저자']

# 학습데이터 
cv = CountVectorizer(tokenizer=okt_pos,stop_words=stopwords)
x_train = cv.fit_transform(x_train)
cv.get_feature_names()
x_train.toarray()

# 테스트데이터
x_test = cv.transform(x_test)
x_test.toarray()

# 분류기 모델
nb = MultinomialNB()
nb.fit(x_train,y_train)

# 테스트 예측
y_predict = nb.predict(x_test)
sum(y_predict == y_test)/360 # 정답률
accuracy_score(y_test,y_predict)

# 혼동행렬
pd.crosstab(y_test,y_predict)
confusion_matrix(y_test,y_predict)
#pd.set_option('display.max_columns',None)
# 정확도, 정밀도, 재현율, f1-score
print(classification_report(y_test,y_predict))

# 분류 클래스
nb.classes_ # 'art_popularculture', 'computer_it', 'economy', 'history_culture','humanities', 'novel', 'poem_essay', 'politics_society', 'religion', 'selfdevelope', 'sience', 'technology_engineering'

# 분류 클래스 갯수
nb.class_count_ 


# 사전확률
import numpy as np
np.exp(nb.class_log_prior_) 

# 분류 클래스별 컬럼의 값의 빈도수
nb.feature_count_

# logistic regression-----------------------------------------------------------------------------------------

x_train,x_test,y_train,y_test = train_test_split(book['text'],book['field'],test_size=0.2)

# 학습데이터 
cv = CountVectorizer(tokenizer=okt_pos,stop_words=stopwords)
x_train = cv.fit_transform(x_train)
x_test = cv.transform(x_test)

lr = LogisticRegression()
lr.fit(x_train,y_train)
lr.classes_

y_pred = lr.predict(x_test)
accuracy_score(y_test,y_pred)
print(classification_report(y_test,y_pred))


정확도 - 나이브베이즈:0.76, 로지스틱회귀:0.74
